# Set up

Load libraries

In [1]:
import os
import sys, getopt
from pathlib import Path
import xml.etree.cElementTree as ET
import pandas as pd

In [2]:
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs

In [3]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# Read inventory 
Read gsheet document with updated inventory

In [4]:
def read_gsheet(SPREADSHEET_ID,RANGE_NAME, JSONFILE):
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(JSONFILE, SCOPES) 
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')


In [5]:
conf_json = Path(os.path.expanduser('~')) / '.secrets/googlecloud/python-gsheet-api.json'
my_doc='105V_zTZ-Hzj1ZuZDt1HFo750viegjmM2zyfPvJmPDF4'
read_gsheet(my_doc,'Assessment Target!A1:AA10000',conf_json)

In [13]:
inventory=pd.DataFrame(values_input[1:], columns=values_input[0])
inventory['Assessment ID']=inventory['Ecosystem ID'].str.replace('_[0-9]+$', '', regex=True)

# Database

In [7]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'iucnecoadm' # we need admin privileges to make changes

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

We need admin privileges to make these changes

In [9]:
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
#qry = """UPDATE rle.assessments set asm_id=%s where asm_id=%s"""
#cur.execute(qry, ('Schaefer_RockyShores_NSW_2020','Schaefer_Rocky_Intertidal_Australia'))
qry = """INSERT INTO ref_list(ref_code,ref_cite) VALUES (%s,%s) ON CONFLICT DO NOTHING"""
cur.execute(qry, ('Roche_EasterScotianShelf_2020','Roche EasterScotianShelf 2020 ???'))
cur.execute(qry, ('Javed_RLE_AbuDhabi_2021','Javed_RLE_AbuDhabi_2021 ???'))
cur.execute(qry, ('Carre_RLE_Madagascar_2020','Carre_RLE_Madagascar_2020 ???'))
cur.execute(qry, ('Rouget_SouthAfrica_2004',' ???'))
cur.execute(qry, ('Obura_etal_CoralReef_WIO_2021',' ???'))
cur.execute(qry, ('Mases-Garcia_OaxacaForests_2021',' ???'))
cur.execute(qry, ('Betancourt_ForestsUruguay_2021',' ???'))
qry = """INSERT INTO rle.assessments(ref_code,asm_id) VALUES (%s,%s) ON CONFLICT DO NOTHING"""
cur.execute(qry, ('Roche_EasterScotianShelf_2020','Roche_EasterScotianShelf_2020'))
cur.execute(qry, ('Javed_RLE_AbuDhabi_2021','Javed_RLE_AbuDhabi_2021'))
cur.execute(qry, ('Carre_RLE_Madagascar_2020','Carre_RLE_Madagascar_2020'))
cur.execute(qry, ('Rouget_SouthAfrica_2004','Rouget_SouthAfrica_2004'))
cur.execute(qry, ('Obura_etal_CoralReef_WIO_2021','Obura_etal_CoralReef_WIO_2021'))
cur.execute(qry, ('Mases-Garcia_OaxacaForests_2021','Mases-Garcia_OaxacaForests_2021'))
cur.execute(qry, ('Betancourt_ForestsUruguay_2021','Betancourt_ForestsUruguay_2021'))

cur.close()
conn.commit()
if conn is not None:
    conn.close()

In [25]:
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
qry = """
SELECT asm_id,eco_id,eco_name,eco_name_orig, eco_name_lang,external_code,assessment_date,
overall_risk_category,risk_category_bounds
FROM rle.assessment_units u
JOIN rle.assessment_overall USING(eco_id)
"""
cur.execute(qry)
assm_units = cur.fetchall()
assm_units_df=pd.DataFrame(assm_units,
                columns=['asm_id','eco_id','eco_name','eco_name_orig', 'eco_name_lang','external_code','assessment_date',
'overall_risk_category','risk_category_bounds'])

conn.commit()
if conn is not None:
    conn.close()


,asm_id,eco_id,eco_name,eco_name_orig,eco_name_lang,external_code,assessment_date,overall_risk_category,risk_category_bounds
0,Carre_RLE_France_2012,Carre_RLE_France_2012:1,Le bassin d’Arcachon,Le bassin d’Arcachon,en,Carre_RLE_France_2012:1,2012-01-01,None,None
1,Carre_RLE_France_2012,Carre_RLE_France_2012:2,Les lacs et étangs du littoral aquitain,Les lacs et étangs du littoral aquitain,en,Carre_RLE_France_2012:2,2012-01-01,None,None
2,Carre_RLE_France_2012,Carre_RLE_France_2012:3,Les lagunes méditerranéennes françaises,Les lagunes méditerranéennes françaises,en,Carre_RLE_France_2012:3,2012-01-01,None,None
3,Carre_RLE_France_2012,Carre_RLE_France_2012:4,Les marais maritimes atlantiques,Les marais maritimes atlantiques,en,Carre_RLE_France_2012:4,2012-01-01,None,None
4,Holdaway_Uncommon_NewZealand_2012,Holdaway_Uncommon_NewZealand_2012:1,Active sand dune,Active sand dune,en,Holdaway_Uncommon_NewZealand_2012:1,2012-01-01,None,None
...,...,...,...,...,...,...,...,...,...
2179,Kontula_FinlandHabitat_2018,RLE-FI-T11.06,T11.06 Mountain acidic and intermediatebasic b...,None,en,RLE-FI-T11.06,2018-10-01,LC,None
2180,Kontula_FinlandHabitat_2018,RLE-FI-T11.07.01,T11.07.01 Acidic and intermediate-basic talus ...,None,en,RLE-FI-T11.07.01,2018-10-01,LC,None
2181,Kontula_FinlandHabitat_2018,RLE-FI-T11.07.02,T11.07.02 Calcareous talus formations,None,en,RLE-FI-T11.07.02,2018-10-01,LC,None
2182,Kontula_FinlandHabitat_2018,RLE-FI-T12.01,T12.01 Canyons,None,en,RLE-FI-T12.01,2018-10-01,LC,None


In [28]:
inv_ss=inventory[inventory['Reference label']=='Keith_Foundations_2013'][['Ecosystem ID','Assessment ID','AT-Name']]

assm_units_df[assm_units_df.asm_id.isin(inv_ss['Assessment ID'])]

,asm_id,eco_id,eco_name,eco_name_orig,eco_name_lang,external_code,assessment_date,overall_risk_category,risk_category_bounds
163,Essl_GermanTamarisk_2013,Essl_CS_GermanTamarisk_2013:1,German tamarisk pioneer vegetation,German tamarisk pioneer vegetation,en,Essl_CS_GermanTamarisk_2013:1,2013-01-01,EN,None
165,Benson_VineThicket_2013,Benson_CS_VineThicketSEAustralia_2013:1,Mock Olive Wilga Peach Bush Carissa dry sub-tr...,Mock Olive Wilga Peach Bush Carissa dry sub-tr...,en,Benson_CS_VineThicketSEAustralia_2013:1,2013-01-01,EN,None
166,Bonifacio_KRSWetland_2013,Bonifacio_CS_KRSWetland_2013:1,Karst rising springs,Karst rising springs,en,Bonifacio_CS_KRSWetland_2013:1,2013-01-01,CR,None
167,Bonifacio_SeagrassCommunity_2013,Bonifacio_CS_SeagrassCommunity_2013:1,Seagrass meadows of South Australia,Seagrass meadows of South Australia,en,Bonifacio_CS_SeagrassCommunity_2013:1,2013-01-01,EN,EN-CR
170,FaberLangendoen_GreatLakesAlvar_2013,FaberLnagendoen_CS_GreatLakesAlvar_2013:1,Great Lakes Alvar,Great Lakes Alvar,en,FaberLnagendoen_CS_GreatLakesAlvar_2013:1,2013-01-01,EN,VU-EN
171,Holdaway_GraniteGravelFields_2013,Holdaway_CS_GraniteGravelFields_2013:1,Granite gravel fields & sandplains,Granite gravel fields & sandplains,en,Holdaway_CS_GraniteGravelFields_2013:1,2013-01-01,LC,None
172,Keith_AralSea_2013,Keith_CS_AralSea_2013:1,Aral Sea,Aral Sea,en,Keith_CS_AralSea_2013:1,2013-01-01,CO,None
173,Keith_GonakierForest_2013,Keith_CS_GonakierForest_2013:1,Gonakier forests of Senegal,Gonakier forests of Senegal,en,Keith_CS_GonakierForest_2013:1,2013-01-01,CR,None
174,Riecken_RaisedBogs_2013,Riecken_CS_RaisedBogs_2013:1,Raised bogs,Raised bogs,en,Riecken_CS_RaisedBogs_2013:1,2013-01-01,CR,None
175,Keith_GiantKelp_2013,Keith_CS_GiantKelp_2013:1,Giant kelp forests,Giant kelp forests,en,Keith_CS_GiantKelp_2013:1,2013-01-01,EN,EN-CR


In [63]:
# 'Shapiro_CongoBasin_2020', 'Roche_EasterScotianShelf_2020'

inventory[inventory['Reference label']=='Kontula_FinlandHabitat_2018'][['Ecosystem ID','Assessment ID','AT-Name','Overall_RC']]



,Ecosystem ID,Assessment ID,AT-Name,Overall_RC
2982,Kontula_FinlandHabitat_2018_1,Kontula_FinlandHabitat_2018,Pending to process,CR
2983,Kontula_FinlandHabitat_2018_2,Kontula_FinlandHabitat_2018,Pending to process,CR
2984,Kontula_FinlandHabitat_2018_3,Kontula_FinlandHabitat_2018,Pending to process,CR
2985,Kontula_FinlandHabitat_2018_4,Kontula_FinlandHabitat_2018,Pending to process,CR
2986,Kontula_FinlandHabitat_2018_5,Kontula_FinlandHabitat_2018,Pending to process,CR
...,...,...,...,...
3365,Kontula_FinlandHabitat_2018_384,Kontula_FinlandHabitat_2018,Pending to process,DD
3366,Kontula_FinlandHabitat_2018_385,Kontula_FinlandHabitat_2018,Pending to process,DD
3367,Kontula_FinlandHabitat_2018_386,Kontula_FinlandHabitat_2018,Pending to process,DD
3368,Kontula_FinlandHabitat_2018_387,Kontula_FinlandHabitat_2018,Pending to process,DD


In [58]:
ins1 = """
INSERT INTO rle.assessment_units (eco_id,eco_name,eco_name_orig, eco_name_lang,external_code)
VALUES %s
ON CONFLICT DO NOTHING
"""
ins2="""
INSERT INTO rle.assessment_overall (asm_id,eco_id,assessment_date,overall_risk_category)
VALUES %s
ON CONFLICT DO NOTHING
"""

ref='Betancourt_ForestsUruguay_2021'
ref='Obura_etal_CoralReef_WIO_2021'
ref='Mases-Garcia_OaxacaForests_2021'
ref='Javed_RLE_AbuDhabi_2021'
ref='Carre_RLE_Madagascar_2020'
ref='Carre_RLE_Madagascar_2020'
inv_ss = inventory[inventory['Reference label']==ref][['Ecosystem ID','Assessment ID','AT-Name','Overall_RC']]


conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
for ss in inv_ss.to_dict(orient='records'):
    cur.execute(ins1,((ss['Ecosystem ID'],ss['AT-Name'],ss['AT-Name'],'sp',ss['Ecosystem ID']),)) # we will need to fix language later
    cur.execute(ins2,((ss['Assessment ID'],ss['Ecosystem ID'],'2021-01-01',ss['Overall_RC']),))
cur.close()
conn.commit()
if conn is not None:
    conn.close()

In [59]:
(ss['Ecosystem ID'],ss['AT-Name'],ss['AT-Name'],'sp',ss['Ecosystem ID'])


('Carre_RLE_Madagascar_2020_36',
 'Le système lagunaire du littoral de Madagascar',
 'Le système lagunaire du littoral de Madagascar',
 'sp',
 'Carre_RLE_Madagascar_2020_36')

[{'Ecosystem ID': 'Betancourt_ForestsUruguay_2021_1',
  'Assessment ID': 'Betancourt_ForestsUruguay_2021',
  'AT-Name': 'Bosque latifoliado de planicie costero',
  'Overall_RC': 'VU'},
 {'Ecosystem ID': 'Betancourt_ForestsUruguay_2021_2',
  'Assessment ID': 'Betancourt_ForestsUruguay_2021',
  'AT-Name': 'Bosque latifoliado de planicie mesofilo',
  'Overall_RC': 'LC'},
 {'Ecosystem ID': 'Betancourt_ForestsUruguay_2021_3',
  'Assessment ID': 'Betancourt_ForestsUruguay_2021',
  'AT-Name': 'Bosque latifoliado de planicie vargedícola',
  'Overall_RC': 'LC'},
 {'Ecosystem ID': 'Betancourt_ForestsUruguay_2021_4',
  'Assessment ID': 'Betancourt_ForestsUruguay_2021',
  'AT-Name': 'Bosque latifoliado serrano metamórfico',
  'Overall_RC': 'LC'},
 {'Ecosystem ID': 'Betancourt_ForestsUruguay_2021_5',
  'Assessment ID': 'Betancourt_ForestsUruguay_2021',
  'AT-Name': 'Bosque latifoliado serrano ígneo',
  'Overall_RC': 'LC'},
 {'Ecosystem ID': 'Betancourt_ForestsUruguay_2021_6',
  'Assessment ID': 'Be